## CS6220: Data Analysis on COVID-19 Vaccine
### Team members: Alex Yang, Shuaizhen Li, Yiman Liu 


## Setup

In [24]:
import pandas as pd
patient_history = pd.read_csv(
    './dataset/2021VAERSDATA.CSV', sep=',', encoding='ISO-8859-1')
patient_sympt = pd.read_csv(
    './dataset/2021VAERSSYMPTOMS.CSV', sep=',', encoding='ISO-8859-1')
patient_vac = pd.read_csv(
    './dataset/2021VAERSVAX.CSV', sep=',', encoding='ISO-8859-1')

C:\Users\zelon\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7,12,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
pd.set_option("display.max_rows", 100)

In [26]:
patient_history.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,NaN,...,None,None,NaN,NaN,2,01/01/2021,NaN,Y,NaN,Pcn and bee venom
1,916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,NaN,...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,NaN,NaN,2,01/01/2021,NaN,Y,NaN,"""Dairy"""
2,916602,01/01/2021,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",NaN,...,None,None,NaN,NaN,2,01/01/2021,NaN,NaN,Y,Shellfish
3,916603,01/01/2021,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",NaN,...,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","got measles from measel shot, mums from mumps ...",NaN,2,01/01/2021,NaN,NaN,NaN,"Diclofenac, novacaine, lidocaine, pickles, tom..."
4,916604,01/01/2021,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",NaN,...,Na,NaN,NaN,NaN,2,01/01/2021,NaN,NaN,NaN,Na


In [27]:
patient_sympt.head()

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,916600,Dysphagia,23.1,Epiglottitis,23.1,NaN,NaN,NaN,NaN,NaN,NaN
1,916601,Anxiety,23.1,Dyspnoea,23.1,NaN,NaN,NaN,NaN,NaN,NaN
2,916602,Chest discomfort,23.1,Dysphagia,23.1,Pain in extremity,23.1,Visual impairment,23.1,NaN,NaN
3,916603,Dizziness,23.1,Fatigue,23.1,Mobility decreased,23.1,NaN,NaN,NaN,NaN
4,916604,Injection site erythema,23.1,Injection site pruritus,23.1,Injection site swelling,23.1,Injection site warmth,23.1,NaN,NaN


In [28]:
patient_vac.tail()

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
34625,1113917,COVID19,MODERNA,01120A,1,OT,NaN,COVID19 (COVID19 (MODERNA))
34626,1113920,COVID19,PFIZER\BIONTECH,Unknown,1,NaN,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))
34627,1113963,COVID19,PFIZER\BIONTECH,EL9264,2,NaN,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))
34628,1115045,COVID19,MODERNA,NaN,2,OT,NaN,COVID19 (COVID19 (MODERNA))
34629,1115348,COVID19,MODERNA,026A21A,1,OT,LA,COVID19 (COVID19 (MODERNA))


### Extract the needed columns from the 3 files and combine them into one dataframe.
* the vaccine symptoms file contains multiple rows for the same vaers_id. This happens when patients have more than 5 symptoms and each row only holds 5. We need to combine the data to single row before merging with the with the other files.  

In [29]:
# get the needed columns from patient history file
patient_history = patient_history.loc[:, [
    'VAERS_ID', 'AGE_YRS', 'SEX', 'CUR_ILL', 'HISTORY', 'ALLERGIES']]

# get the needed columns from patient vaccine file
patient_vac = patient_vac.loc[:, ['VAERS_ID', 'VAX_NAME']]

# commbining patient's vaccine symptoms to a single cell
patient_sympt['SYMPTOMS'] = patient_sympt['SYMPTOM1'].fillna('')+',' + patient_sympt['SYMPTOM2'].fillna(
    '')+',' + patient_sympt['SYMPTOM3'].fillna('')+',' + patient_sympt['SYMPTOM4'].fillna('')+',' + patient_sympt['SYMPTOM5'].fillna('')
patient_sympt = patient_sympt.groupby(by='VAERS_ID')['SYMPTOMS'].agg(
    lambda x: ','.join(x)).reset_index()

In [30]:
# merge all extracted info and combine them to a single dataframe

df = patient_history.merge(patient_vac, on='VAERS_ID')
df = pd.merge(df, patient_sympt,  left_on='VAERS_ID', right_on='VAERS_ID')

In [31]:
df.head()

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS
0,916600,33.0,F,None,None,Pcn and bee venom,COVID19 (COVID19 (MODERNA)),"Dysphagia,Epiglottitis,,,"
1,916601,73.0,F,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,"""Dairy""",COVID19 (COVID19 (MODERNA)),"Anxiety,Dyspnoea,,,"
2,916602,23.0,F,None,None,Shellfish,COVID19 (COVID19 (PFIZER-BIONTECH)),"Chest discomfort,Dysphagia,Pain in extremity,V..."
3,916603,58.0,F,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom...",COVID19 (COVID19 (MODERNA)),"Dizziness,Fatigue,Mobility decreased,,"
4,916604,47.0,F,Na,NaN,Na,COVID19 (COVID19 (MODERNA)),"Injection site erythema,Injection site pruritu..."


# Data Preprocessing 

In [32]:

# remove NaN Value
df = df.fillna(value="")
# prepare y values step1
# make symptom columns into a list
df['SYMPTOMS'] = df['SYMPTOMS'].apply(lambda x: x.split(","))
# remove any empty values in the list.
df['SYMPTOMS'] = df['SYMPTOMS'].apply(lambda x: [i for i in x if i])
df = df.drop_duplicates(['VAERS_ID'])

In [33]:
def convertAge(string_age):
    if string_age == "":
        return "unknown_age"
    if 0 <= float(string_age) <= 2:
        return "0-2 yr"
    if 3 <= float(string_age) <= 12:
        return "3-12 yr"
    if 13 <= float(string_age) <= 19:
        return "13-19 yr"
    if 20 <= float(string_age) <= 39:
        return "20-39 yr"
    if 40 <= float(string_age) <= 59:
        return "40-59 yr"
    return "over_60 yr"


def changeVaxName(vax_name):
    if 'MODERNA' in vax_name:
        return 'MODERNA'
    if 'PFIZER' in vax_name:
        return 'PFIZER'

    return 'JANSSEN'

In [34]:
# remove none coivd related samples
df = df[df['VAX_NAME'].str.contains(
    '|'.join(['MODERNA', 'PFIZER-BIONTECH'])) & df['VAX_NAME'].isna() == False]
# Bucketize age 
df['AGE_CAT'] = df['AGE_YRS'].apply(lambda x: convertAge(x))
# simmplify vaccine names
df['VAX_NAME'] = df['VAX_NAME'].apply(lambda x: changeVaxName(x))

In [35]:
# words and phrase to be removed from the dataset
remove_word_list = ['none', 'na', 'nkda', 'nka', 'unknown', 'medical history/concurrent conditions:', 'no adve...', 'no known allergies', 'no illnesses at time or prior', 'no allergies', 'no adverse event (no reported medical history)', 'comments: list of non-encoded patient relevant...', 'ukn', ' no adverse event (no reported medical history.)',
                    ' no adverse event (no medical history reported)',
                    ' no adverse event (no adverse event was reported.)',
                    ' no adverse event (no medical history reported.)', '',
                    ' no adverse event',
                    ' no adverse event (no patient history reported.)',
                    ' no adverse event (medical history not provided)',
                    ' no adverse event (medical history not reported)',
                    ' no adverse event (no adverse event history)',
                    ' no adverse event (no medical history provided)',
                    ' no adverse event (no reported adverse event)',
                    ' no adverse event (no recorded medical history.)',
                    ' no adverse event (no medical history provided.)',
                    'denies', 'denies any illness at time of vaccition or up ...', 'denies any illnesses', 'patient denies', 'unkown', 'no known', 'comments: list of non-encoded patient relevant...', 'no illnesses at time of vaccition', 'no illness one month prior to or at time of va...']


def remove_words(strings):
    for words in remove_word_list:
        strings = strings.replace(words, '')
    return strings

In [36]:
# make the health_condition text lower case
df['CUR_ILL'] = df['CUR_ILL'].str.lower()
df['HISTORY'] = df['HISTORY'].str.lower()
df['ALLERGIES'] = df['ALLERGIES'].str.lower()
# remove list of words from columns

df['CUR_ILL'] = df['CUR_ILL'].apply(lambda x: remove_words(x))
df['HISTORY'] = df['HISTORY'].apply(lambda x: remove_words(x))
df['ALLERGIES'] = df['ALLERGIES'].apply(lambda x: remove_words(x))


 

#### We need to extract meaningful keywords related to patients' health condition. So we are concatenating the text in the column "cur_ill", "history",  "Allergy". and store the concatenated values in the column " HEALTH_CONDITION" .   

In [37]:

# combining patients' current illness , medical history and, allergies and store in the "health condition column"
df['HEALTH_CONDITION'] = df['CUR_ILL']+"," + df['HISTORY']+","+df['ALLERGIES']

#### The text in the Health conditon column contain words unrelated to patients'  health. It also contains words in differents forms and misspellings. We need to perform the follows to clean the data. 

In [38]:
import nltk
import itertools
from string import punctuation
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [39]:
df.head()

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS,AGE_CAT,HEALTH_CONDITION
0,916600,33.0,F,,,pcn and bee venom,MODERNA,"[Dysphagia, Epiglottitis]",20-39 yr,",,pcn and bee venom"
1,916601,73.0,F,patient residing at nursing facility. see pati...,patient residing at nursing facility. see pati...,"""dairy""",MODERNA,"[Anxiety, Dyspnoea]",over_60 yr,patient residing at nursing facility. see pati...
2,916602,23.0,F,,,shellfish,PFIZER,"[Chest discomfort, Dysphagia, Pain in extremit...",20-39 yr,",,shellfish"
3,916603,58.0,F,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","diclofec, novacaine, lidocaine, pickles, tomat...",MODERNA,"[Dizziness, Fatigue, Mobility decreased]",40-59 yr,"kidney infection,diverticulitis, mitral valve ..."
4,916604,47.0,F,,,,MODERNA,"[Injection site erythema, Injection site pruri...",40-59 yr,",,"


In [40]:
def remove_words_containning_numbers(word):
    if word.isalpha():
        return word
    return ""

#### tokenize the text in health_condition column.  There are 16361 unique words in the column. 

In [41]:
# tokenize df_training['HEALTH_CONDITION']
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: nltk.word_tokenize(x))
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

16361

In [42]:
# remove any punctuations from words
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [''.join(c if c not in punctuation else " " for c in s) for s in x])
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

16076

In [43]:
# remove any words mix with number
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [remove_words_containning_numbers(w) for w in x])
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

10565

In [44]:
# remove any empty words
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [string for string in x if string != ""])
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

10564

In [45]:
# remove stop words
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [words for words in x if words not in stopwords])
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

10433

In [46]:
# extract only nouns form the Health_conditon column.
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [words for words, pos in nltk.pos_tag(x) if pos == "NN" or pos == 'NNS'])

allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

7237

In [47]:
# auto correct spelling errors (run paralle)
import multiprocessing
from autocorrect import Speller
import time
spell = Speller(lang='en')


import mypack.correc as co
import numpy as np
new_dff=df.loc[:,'HEALTH_CONDITION']
new_dff
start_time = time.time()
pool = multiprocessing.Pool(multiprocessing.cpu_count())

new_dff=new_dff.apply(lambda x :  pool.map(co.correction, x ))
   
print("--- %s seconds ---" % (time.time() - start_time))

location
--- 2323.7956500053406 seconds ---


In [48]:
df['HEALTH_CONDITION']=new_dff

In [54]:
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
allwords_list = list(allwords)
allwords = sorted(allwords_list)
len(allwords)


6312

#### Analyze which words are similar and group them together 

In [55]:
from difflib import SequenceMatcher

# function to check how similar the words are
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

word_grouping=allwords.copy()

# if the words are more than 80% similar, put them in a list
dic_word=[]
for a in word_grouping:
    
    group=[]
    for b in word_grouping:
        
        if similar(a, b)> 0.80:
            
            group.append (b)
            word_grouping.remove(b)
    dic_word.append(group)
   # word_grouping.remove(word_grouping[a])


In [65]:
print(dic_word)

[['aaa'], ['abd'], ['ability', 'disability'], ['abnormal', 'abnormality', 'normal'], ['aboutus', 'bouts'], ['abrasions'], ['abscess'], ['absent'], ['absorption', 'malabsorption'], ['abuse'], ['academia'], ['accident'], ['ace', 'acne', 'ance', 'face', 'pace'], ['acetaminophen', 'acetiminophen'], ['acetominiphen'], ['acf'], ['achieves'], ['acidity'], ['acitretin'], ['acre'], ['actenol'], ['active'], ['activity'], ['actor', 'factor'], ['acyclovir', 'alcyclovir'], ['add', 'adds'], ['addiction'], ['additives'], ['adenocarcinoma'], ['adenoma'], ['adenosine'], ['adh'], ['adhesive'], ['adm'], ['administration'], ['admit'], ['admitting'], ['adenomyosis', 'adneomyosis'], ['advance'], ['ae'], ['aesthetics'], ['affair'], ['affect', 'effects'], ['afluria'], ['africa'], ['afterward'], ['agar'], ['agency'], ['agent'], ['aggregation'], ['aggression'], ['agonist'], ['agranulocytosis', 'granulomatosis'], ['agreement'], ['acid', 'aid', 'aids', 'arid', 'paid', 'said'], ['airborne'], ['airspace'], ['airway

####  Normalize the words 

In [57]:
simmilar_words = [{'abnormal': ['abnormal', 'abnormality']}, {'acetaminophen': ['acetaminophen', 'acetiminophen']}, {'acyclovir': ['acyclovir', 'alcyclovir']}, {'add': ['add', 'adds']}, {'adenomyosis': ['adenomyosis', 'adneomyosis']}, {'affect': ['affect', 'effects']}, {'agranulocytosis': ['agranulocytosis', 'granulomatosis']}, {'acid': ['acid', 'aid', 'aids']}, {'alcohol': ['alcohol', 'alcohols']}, {'allergy': ['allergy', 'allergies']}, {'alzheimers': ['alheimers', 'alxheimer', 'alzheimers']}, {'amitriptyline': ['amatriptoline', 'amitriptyline']}, {'amoxicillin': ['amoxacillin', 'amoxapine', 'amoxcillin', 'amoxiccillin', 'amoxicillan', 'amoxicillen', 'amoxicilliin', 'amoxicillin', 'amoxillion', 'amoxocin', 'amoxycillan', 'amoxycillian', 'amoxycillin']}, {'abnormalities': ['abnormalities', 'anomalies']}, {'anticoagulant': ['anticoagulant', 'anticoagulation']}, {'ant': ['ant', 'ants']}, {'azithromycin': ['arithimycin', 'arrhthromycin', 'arrythromycin', 'arythomycin', 'azithromycin', 'azithromyocin', 'azythromycin', 'zithromycin']}, {'erthromycin': ['erthromycin', 'erythromycin']}, {'arteriosclerosis': ['arteriosclerosis', 'atherosclerosis', 'otosclerosis']}, {'lovastatin': ['atorvastat', 'atovastatin', 'lovastatin', 'rovastatin']}, {'avocado': ['avocado', 'avocados']}, {'awake': ['awake', 'wake']}, {'bacitracin': ['bacitracin', 'bacitrian', 'bactiracin']}, {'ban': ['ban', 'bans']}, {'bee': ['bee', 'bees']}, {'behavior': ['behavior', 'behaviors']}, {'benzodiazepines': ['benzodiazapenes', 'benzodiazepines']}, {'berries': ['berries', 'bluberries']}, {'breast': ['breast', 'breasts']}, {'bronchiolitis': ['brochitis', 'bronchiolitis']}, {'ibuprofen': ['buprofen', 'ibuprofen']}, {'bypass': ['bypass', 'passes']}, {'calculation': ['calculation', 'circulation', 'coagulation']}, {'cauliflower': ['cauliflower', 'rawcauliflower']}, {'cephalexin': ['cefalaxin', 'cephalexin']}, {'cefuroxime': ['cefuroxime', 'cefuroxme']}, {'center': ['center', 'enter', 'enters']}, {'cephalosporin': ['cefalosporins', 'celphlasporins', 'cephalasporins', 'cephalosporin', 'cephalosporis', 'cephlasporins']}, {'chlorhexidine': ['chlohexidine', 'chlorhexidine', 'cholerahexadine', 'cholorehexadine']}, {'cholecystitis': ['chocystitis', 'cholecystitis']}, {'cholesterol': ['cholesterol', 'clolestroal', 'colorestrol', 'hypercholesterol']}, {'ciprofloxacin': ['ciproflaxen', 'ciprofloxacin']}, {'lincomycin': ['clindamiacin', 'clindomyacin', 'clyndomycin', 'lincomycin']}, {'clinoril': ['clinoril', 'linsopril']}, {'lorazepam': ['clozepam', 'lorazepam']}, {'color': ['color', 'colors']}, {'complain': ['complain', 'complaints']}, {'compassion': ['compassion', 'compression', 'impression']}, {'concentrate': ['concentrate', 'contraction']}, {'condition': ['condition', 'conditioning']}, {'conjunctivitis': ['conjunctivitis', 'rhinoconjunctivitis']}, {'oropharyngeal': ['oropharyngeal', 'sopharyngeal']}, {'cyclobenzaprine': ['cyclobenzaprine', 'cyclobezprine']}, {'hysteroscopy': ['cystoscopy', 'hysteroscopy']}, {'deaf': ['deaf', 'def']}, {'doxycycline': ['doxicillin', 'doxicycline', 'doxocyclin', 'doxycline', 'doxycyclin', 'doxycycline', 'doxycyclineusea', 'doxycylcine', 'doxycyline', 'doxylamine']}, {'deficiency': ['deficiency', 'deficit']}, {'demerol': ['demarol', 'demerol']}, {'dependence': ['dependence', 'dependent']}, {'dexamethasone': ['betamethasone', 'deamethasone', 'dexamethasone']}, {'diabetic': ['diabetic', 'prediabetic']}, {'diagnoses': ['diagnoses', 'diagnostics']}, {'dilaudid': ['dialudid', 'diaudid', 'dilaudid']}, {'diverticulosis': ['diberticulosis', 'divertculosis', 'diverticulosis']}, {'difficult': ['difficult', 'difficulty']}, {'dioxide': ['dioxide', 'oxide']}, {'dis': ['dis', 'disc', 'dish']}, {'lipidemia': ['dislipidemia', 'dyslipemia', 'dysplipdemia', 'lipidemia']}, {'diuresis': ['diuresis', 'duragesis']}, {'draw': ['draw', 'draws']}, {'drop': ['drop', 'drops']}, {'dysphonia': ['dysphonia', 'dysponia']}, {'eastern': ['eastern', 'eaten', 'stern']}, {'egg': ['egg', 'eggs']}, {'neomycin': ['emyacin', 'neomycin']}, {'epicondylitis': ['epicondylitis', 'spondylitis']}, {'esophagitis': ['esofagitis', 'esophagitis', 'esphagitis', 'oesophagitis']}, {'exacerbation': ['exacerbation', 'exascerbation', 'exhacerbation']}, {'eye': ['eye', 'eyes']}, {'fascists': ['fasciatus', 'fascists']}, {'fat': ['fat', 'fats']}, {'feel': ['feel', 'feels']}, {'find': ['find', 'finds']}, {'fir': ['fir', 'fires']}, {'flights': ['flights', 'light', 'lights']}, {'fluorescent': ['fluorescent', 'fluoroscein']}, {'folliculitis': ['foliculitis', 'folliculitis']}, {'form': ['form', 'forms']}, {'fosamax': ['fosamax', 'fosomax']}, {'gastrectomy': ['gastrectomy', 'gastrostomy']}, {'gastroesophageal': ['gastroesophageal', 'gastroesphageal']}, {'gadolinium': ['gadolinium', 'glalinium']}, {'hazelnuts': ['hazeenut', 'hazelnuts']}, {'head': ['head', 'heads']}, {'hemochromatosis': ['hemachromatosis', 'hemochromatosis']}, {'hemodialysis': ['hemodialiysis', 'hemodialysis', 'hemolysis']}, {
    'hypoglycemia': ['hipoglicemia', 'hypoglycemia']}, {'horse': ['horse', 'horses']}, {'hydronephrosis': ['hydonephrosis', 'hydronephrosis']}, {'hydralazine': ['hydralazine', 'hydrazaline']}, {'dehydration': ['dehydration', 'hydration']}, {'hydromorphone': ['hidromorphine', 'hydromorphone']}, {'hydroxy': ['hydro', 'hydroxy']}, {'hyperthyroidism': ['hyopothroidism', 'hyperthyroidism', 'hypohyroidism', 'hypoparathyroidism', 'hyporthyroidism', 'hypothoidism', 'hypothroid', 'hypothryoidism', 'hypothryroidism', 'hypothyriodism', 'hypothyroidism', 'hypothyroiditis', 'hypothyroigism', 'hypthyroidism']}, {'hyperlipidaemia': ['hypelipidemia', 'hyperdyslipidemia', 'hyperlidemia', 'hyperlipademia', 'hyperlipedemia', 'hyperlipidaemia', 'hyperlipidem', 'hyperlipidemia', 'hyperlipidimia', 'hyperlipiemia', 'hypolipidemia', 'hyprlipidemia']}, {'hypercalcemia': ['hpercalcemia', 'hypercalcemia', 'hyperglycemia', 'hyperkalemia']}, {'hypercholesterolemia': ['hyperchloestermia', 'hypercholeroltemia', 'hypercholesteralemia', 'hypercholestermia', 'hypercholesterolemia', 'hypercholesterolism', 'hypercholsterolemia', 'hyperholesteremia', 'hypersholesteremia', 'hypocholesteremia', 'ypercholesteremia', 'hypercolestoramia']}, {'hyperhidrosis': ['hyperhidrosis', 'hyperhydrosis', 'hyperthyrodism']}, {'hypermetropia': ['hypermetropia', 'hyperopia']}, {'hyperparathyroidism': ['hyperparathyroidism', 'hypertyroidism', 'hypoparathyroidim']}, {'hypertension': ['hypertension', 'prehypertension']}, {'hypovolemia': ['hypocholemia', 'hypovolemia']}, {'hypoglycemic': ['hypoglycemic', 'hypooglycemia']}, {'hypokalemia': ['hyokalemia', 'hypokalemia']}, {'hypoventilation': ['hypoventilation', 'ventilation']}, {'inflammation': ['inflammation', 'inflammatory']}, {'inhibition': ['inhibition', 'inhibitors']}, {'laminectomy': ['hemilaminectomy', 'laminectomy']}, {'laryngitis': ['laryngitis', 'pharyngitis']}, {'leaves': ['leaves', 'level']}, {'lip': ['lip', 'lips']}, {'list': ['list', 'lists']}, {'lymphedema': ['lymphadem', 'lymphedema']}, {'lymphoid': ['lymphnode', 'lymphoid']}, {'macrodantin': ['macrodantin', 'microdantin']}, {'macrocytosis': ['macrocytosis', 'mastocytosis', 'stomatocytosis']}, {'menopausal': ['menopausal', 'perimenopausal', 'postmenopausal']}, {'menorrhagia': ['menometrorrhagia', 'menorrhagia']}, {'metabolic': ['hypometabolism', 'metabolic']}, {'albuminuria': ['albuminuria', 'microalbuminuria']}, {'minocycline': ['minicycline', 'minocycline']}, {'monistat': ['monistat', 'monostat']}, {'myocarditis': ['myocarditis', 'myopericarditis']}, {'neosporin': ['neospin', 'neosporin']}, {'nephritis': ['nephritis', 'neuritis', 'polynephritis']}, {'neuron': ['neuro', 'neuron', 'neurons']}, {'neurofibromatosis': ['fibromatosis', 'neurofibromatosis']}, {'sarcoidosis': ['neurosarcoidosis', 'sarcoidosis']}, {'night': ['night', 'nights']}, {'nitrofuran': ['nitrofuran', 'nitrofurantoin']}, {'melanoma': ['melanoma', 'nonmelanoma']}, {'nut': ['nut', 'nuts']}, {'occasion': ['occasion', 'occlusion']}, {'orbital': ['orbital', 'orita']}, {'ephedrine': ['ephedrine', 'orphedrine']}, {'osteopetrosis': ['oesteoporosis', 'osteonecrosis', 'osteopetrosis', 'osteoporosis', 'osteporosis', 'ostprosis']}, {'palpitations': ['palpatation', 'palpitatioins', 'palpitations']}, {'parkinson': ['parkinson', 'parkinsonisms']}, {'pepper': ['pepper', 'peppers']}, {'laryngeal': ['laryngeal', 'pharyngeal']}, {'phenergan': ['phenergan', 'phergergan']}, {'phenobarbitals': ['phenobarb', 'phenobarbitals']}, {'pig': ['pig', 'pigs']}, {'pistachio': ['pistachio', 'pistschios']}, {'place': ['place', 'places']}, {'plan': ['plan', 'plans']}, {'polyarteritis': ['polyarteritis', 'polyarthritis']}, {'preservatives': ['perservatives', 'preservatives']}, {'propoxyphene': ['propoxyphene', 'propyphene']}, {'pseudoephedrine': ['psedophedrine', 'pseudoephedrine', 'pseudophendirine']}, {'quadriplegia': ['quadraplegic', 'quadriplegia']}, {'question': ['question', 'questions']}, {'benzalclonium': ['benzalcloniumbenzalkonium', 'benzene', 'benzoate', 'benzocaine', 'benzodiazepine', 'benzodiazepines', 'benzoid', 'benzoly', 'benzoperoxide', 'benzoyl']}, {'bronchiectasis': ['bronchial', 'bronchiectasis', 'bronchiolitis', 'bronchitis', 'bronchopneumonia', 'bronchospasm']}, {'cranberries': ['cranberries', 'raspberries']}, {'result': ['result', 'results']}, {'rheumatoid': ['rheumatiod', 'rheumatoid']}, {'run': ['run', 'runs']}, {'salicylate': ['salicylate', 'salicylicates']}, {'thrombocytopemia': ['thrombocytopemia', 'thrombocytopenia', 'thrombocytosis', 'thromboembolism', 'thrombophilia', 'thrombophlebitis', 'thrombus']}, {'ischemia': ['ischemia', 'schema']}, {'shellfish': ['selfish', 'shellfish']}, {'shoot': ['shoot', 'shot']}, {'sign': ['sign', 'signs']}, {'penicillins': ['pencillers', 'peneciline', 'penicillin', 'penillicillins', 'pennicellins']}, {'sinusitis': ['sinnusitis', 'sinusitis', 'tinnitis']}, {'sleep': ['sleep', 'sleeps']}, {'smoke': ['smoke', 'smokers']}]

In [58]:
simmilar_words2 = [{'hypogammaglobulinemia': ['ypogammaglobuliemia', 'hypogammaglobulinemia', 'ypogammaglobulonemia']}, {'spironolactone': ['spironolactone', 'sprionolactone']}, {'spondylolisthesis': ['spondylolisthesis', 'spondylolysis', 'spondylosis', 'spondylolithesis', 'spondylopathies']}, {'stay': ['stay', 'stays']}, {'sulfanilamide': ['sulfailamide', 'sulfanilamide']}, {'sulfide': ['sulfide', 'sulfomides']}, {'supplement': ['supplement', 'supplements']}, {'tachycardia': ['tacacardia', 'tachcardia', 'tachycardia']}, {'tetracyclines': ['tetracicline', 'tetracyclines']}, {'thioridazine': ['thioridazine', 'thorazine']}, {'thrombocytosis': ['thrombocytosis', 'thrombosis']}, {'issues': ['issues', 'tissues']}, {'titan': ['titan', 'titans']}, {'tobramycin': ['tobdramycin', 'tobramycin']}, {'tolectin': ['tolectin', 'tolmetin']}, {'toxicity': ['ototoxicity', 'toxicity']}, {'transplant': ['transplant', 'transplants']}, {'tube': ['tube', 'tubes']}, {'onions': ['onions', 'union']}, {'urticaria': ['uritcaria', 'urticaria', 'uticaria']}, {'value': ['value', 'valves']}, {'vancomycin': ['vancomycin', 'vancomyocin']}, {'atrioventricular': ['atrioventricular', 'ventricular']}, {'visit': ['visit', 'visits']}, {'week': ['week', 'weeks']}, {'well': ['well', 'wells']}, {'abnormal': ['abnormal', 'abnormality']}, {'acetaminophen': ['acetaminophen', 'acetaminophine', 'acetate', 'acetominiphen']}, {'abscess': ['abscess', 'abscessus']}, {'absence': ['absence', 'absent']}, {'adenocarcinoma': ['adenocarcinoma', 'adenoma', 'adenomyosis', 'adenosine']}, {'adhesion': ['adhesion', 'adhesive']}, {'admin': ['admin', 'administration']}, {'admit': ['admit', 'admits']}, {'agenesis': ['agenesis', 'agenisis']}, {'albumin': ['albumin', 'albuminuria']}, {'alergies': ['alergies', 'allegergies', 'allegeries', 'allegies', 'allegra', 'allergans', 'allergen', 'allergic', 'allergiers', 'allergiex', 'allergist', 'allergy', 'alleriges']}, {'aluminium': ['aluminium', 'aluminum']}, {'alzheimer': ['alzheimer', 'alzheimers']}, {'amblyopia': ['amblyopia', 'ambulance', 'ambulatory']}, {'aminoglycosides': ['aminoglycosides', 'aminooglycosides']}, {'amiodarone': ['amiodarone', 'amioderone']}, {'amitriptyline': ['amitriptyline', 'amitryptiline']}, {'amlodipine': ['amlodipine', 'amlodopine']}, {'amox': ['amox', 'amoxi', 'amoxicillin', 'amoxil']}, {'ampicillin': ['ampicillin', 'ampicillinhives']}, {'amputaion': ['amputaion', 'amputation']}, {'anemia': ['anemia', 'anemic', 'anemica']}, {'anesthesia': ['anesthesia', 'anesthessia', 'anesthetic']}, {'aneurism': ['aneurism', 'aneursym', 'aneurysm']}, {'angeoedema': ['angeoedema', 'angio', 'angiodema', 'angiodysplasia', 'angioedema', 'angiogram', 'angiomyolipoma', 'angiopathy', 'angioplasty', 'angiotensin', 'angloidosis']}, {'antibiotic': ['antibiotic', 'antibiotoics', 'antibitotics', 'antibody', 'antiboitics']}, {'anticoag': ['anticoag', 'anticoagulant']}, {'anuerysm': ['anuerysm', 'anurism', 'anursym']}, {'anxiety': ['anxiety', 'anxity']}, {'aphagia': ['aphagia', 'aphalaxic', 'aphalaxis', 'aphalxis', 'aphalyxis', 'aphasia']}, {'aphylaxes': ['aphylaxes', 'aphylaxis', 'aphylaxtis']}, {'apnea': ['apnea', 'apneia', 'apnoea']}, {'appendectomy': ['appendectomy', 'appendiceal', 'appendicitis', 'appendix']}, {'arrhythmia': ['arrhythmia', 'arrythmia']}, {'arteriole': ['arteriole', 'arteriosclerosis', 'arteritis', 'artery', 'arthirits', 'arthitis', 'arthralgia', 'arthria', 'arthrities', 'arthritis', 'arthritus', 'arthrodesis', 'arthrofibrosis', 'arthropathy', 'arthroplasty', 'arthropod', 'arthroscopy', 'arthrrities', 'artritis']}, {'asbestos': ['asbestos', 'asbestosis']}, {'aasthma': ['aasthma', 'ashma', 'ashtma', 'asma', 'asthma', 'asthmatic', 'asthmaticus', 'athsma']}, {'asparagus': ['asparagus', 'aspartame', 'aspartane', 'astio', 'astomosis']}, {'aspercream': ['aspercream', 'asperger', 'aspergers', 'aspertane', 'aspertein', 'aspirin', 'aspirne', 'atelol', 'atenolol', 'atenolot']}, {'asplenia': ['asplenia', 'asplenism', 'augementin', 'augmentation', 'augmenten', 'augmentin', 'augmenton']}, {'avacado': ['avacado', 'avacodo', 'avacodos', 'avocado', 'avocadoes']}, {'avalox': ['avalox', 'avelox', 'azithromiacine', 'azithromycin', 'azithromyecin', 'azythramycin', 'bacteremia', 'bacteria', 'bacterium', 'bactrain', 'bactria', 'bactrim', 'bactrin', 'bactrium', 'bactriun', 'bactroban', 'bactruim', 'bactrum']}, {'barret': ['barret', 'barrets', 'barrett', 'barretts']}, {'benzalclonium': ['benzalclonium', 'benzalkonium']}, {'benzo': ['benzo', 'benzodiazepine', 'benzoin', 'benzoine', 'benzol', 'benzotate', 'benzote']}, {'bezapril': ['bezapril', 'bezepril']}, {'biacin': ['biacin', 'biactin']}, {'biaxcin': ['biaxcin', 'biaxin']}, {'bicep': ['bicep', 'biceps']}, {'bifid': ['bifid', 'bifida']}, {'bleed': ['bleed', 'bleeding', 'bleeds']}, {'blind': ['blind', 'blindness']}, {'bronchiectasis': ['bronchiectasis', 'bronchiolitis', 'bronchus']}, {'bupropion': ['bupropion', 'buproprion']}, {'butyl': ['butyl', 'butylcarbamate']}, {'candida': [
    'candida', 'candidate', 'candidiasis']}, {'cantalope': ['cantalope', 'cantaloupe', 'canteloupe']}, {'capsaicin': ['capsaicin', 'capsasin']}, {'carbamate': ['carbamate', 'carbamaze', 'carbamazepine', 'carbamazepone']}, {'carcinoid': ['carcinoid', 'carcinoma', 'carcinosarcoma']}, {'cardia': ['cardia', 'cardiac', 'cardio', 'cardiologist', 'cardiology', 'cardiomediastil', 'cardiomegaly', 'cardiomyopathy', 'cardiorel', 'cardiovascular', 'cardioversion', 'cardioversions']}, {'cefaclor': ['cefaclor', 'cefador', 'cefadroxil', 'cefalexin', 'cefazolin']}, {'cefdenir': ['cefdenir', 'cefdiner', 'cefdinir', 'cefdnir', 'cefnidir']}, {'cefepime': ['cefepime', 'cefipime']}, {'celluitis': ['celluitis', 'cellulitis']}, {'cephalaxin': ['cephalaxin', 'cephalexin', 'cephalon', 'cephalosporin', 'cephasporins']}, {'cephlex': ['cephlex', 'cephlexin', 'ceplalaxin']}, {'cerebri': ['cerebri', 'cerebritis', 'cerebro', 'cerebrovasc', 'cerebrovascular', 'cerebyx']}, {'cerumen': ['cerumen', 'cerumenex']}, {'chemo': ['chemo', 'chemoradiation', 'chemotherapy', 'chemothrapy']}, {'chlor': ['chlor', 'chlorahexidine', 'chloramphenicol', 'chloraprep', 'chlorascrub', 'chlordiazepoxide', 'chloresterol', 'chlorhexidine', 'chloride', 'chlorine', 'chlormycetin', 'chloroprep', 'chloroquine', 'chlorothiazide', 'chloroxylenol', 'chlorpheniramine', 'chlorpromazine', 'chlorthalidone']}, {'chocholate': ['chocholate', 'chocolate']}, {'cholecystectomy': ['cholecystectomy', 'cholecystitis', 'choledocholithiasis', 'cholelithiasis']}, {'chloesterol': ['chloesterol', 'cholersterol', 'cholesteral', 'cholesterol', 'cholesterolemia', 'cholesterolm', 'cholestrol', 'cholestyramine']}, {'cillin': ['cillin', 'cillins']}, {'ciprodex': ['ciprodex', 'ciproflaxin', 'ciprofloxaci', 'ciprofloxacin', 'ciprofloxin']}, {'clarithomycin': ['clarithomycin', 'clarithromycin']}, {'clindamayacin': ['clindamayacin', 'clindamyacin', 'clindamyaxin', 'clindamycin', 'clindamycinrash', 'clindamysin', 'clindaymyacin', 'clindomycin']}, {'colities': ['colities', 'colitis', 'colits']}, {'colonoscopy': ['colonoscopy', 'colonscopy']}, {'compazene': ['compazene', 'compazine', 'compezine']}, {'congestion': ['congestion', 'congestive']}, {'conjuctivits': ['conjuctivits', 'conjunctivitis', 'conjuntivitis']}, {'constiipation': ['constiipation', 'constipation']}, {'costichondritis': ['costichondritis', 'costochondritis', 'costochronitis']}, {'cyclobenzaprine': ['cyclobenzaprine', 'cyclobenzoprine', 'cyclodrine', 'cyclosporin', 'cyclosporine']}, {'dabigatran': ['dabigatran', 'dabigitran']}, {'defibrilator': ['defibrilator', 'defibrillator', 'defibulator']}, {'deficency': ['deficency', 'deficience', 'deficiency', 'deficient', 'deficit']}, {'demand': ['demand', 'demenita', 'dementia', 'dementio']}, {'depacote': ['depacote', 'depakote']}, {'depatale': ['depatale', 'dependence', 'dependency']}, {'dermatofibroma': ['dermatofibroma', 'dermatographia', 'dermatographism', 'dermatologist', 'dermatomyocitis', 'dermatomyositis', 'dermatophytosis', 'dermatoymositis']}, {'diabertes': ['diabertes', 'diabet', 'diabetes', 'diabetic', 'diabeties', 'diabetis', 'diaibetes']}, {'diahhrea': ['diahhrea', 'diahrea', 'diarreha', 'diarrhea', 'diarrhoea', 'dibetes']}, {'dicloxacilliam': ['dicloxacilliam', 'dicloxacillin', 'dicloxicillin']}, {'diphenhyd': ['diphenhyd', 'diphenhydramine']}, {'diverculitis': ['diverculitis', 'diversion', 'diverticulas', 'diverticulitis', 'diverticulosis', 'diverticululitis', 'diverticulum', 'divertivulitis']}, {'dizziness': ['dizziness', 'dizzy']}, {'dupuytrans': ['dupuytrans', 'dupuytren', 'dupuytrens']}, {'duricef': ['duricef', 'duriceff']}, {'dustmite': ['dustmite', 'dustmites']}, {'dyspahgia': ['dyspahgia', 'dyspepsia', 'dysplasia']}, {'dysphagia': ['dysphagia', 'dysphonia', 'dysphoria', 'dysphoric']}, {'dysrhythmia': ['dysrhythmia', 'dysthymia']}, {'emphezema': ['emphezema', 'emphysema']}, {'encephalitis': ['encephalitis', 'encephalopathy']}, {'endocin': ['endocin', 'endocrine', 'endocrinologist', 'endocrinology']}, {'endometeriosis': ['endometeriosis', 'endometriosis', 'endometropsis', 'endometrosis', 'endrometriosis']}, {'enviormental': ['enviormental', 'environment']}, {'eosinophil': ['eosinophil', 'eosinophilia']}, {'epicondyle': ['epicondyle', 'epicondylitis']}, {'erythema': ['erythema', 'erythematosis', 'erythematosus']}, {'erythmycin': ['erythmycin', 'erythomycin', 'erythritol', 'erythrmycin']}, {'erythromelalgia': ['erythromelalgia', 'erythromiacin', 'erythromicin', 'erythromyacin', 'erythropoietin', 'erytrhomycin']}, {'esophageal': ['esophageal', 'esophagectomy', 'esophagitis', 'esophagus', 'esophogus', 'esphogael']}, {'ethyl': ['ethyl', 'ethylenediamine']}, {'fatigue': ['fatigue', 'fatique']}, {'fasciatis': ['fasciatis', 'fasciculation', 'fasciitis', 'fascisitis', 'fascitis']}, {'fentanol': ['fentanol', 'fentanyl', 'fentayl', 'fentyl', 'fentynol']}, {'fibirllation': ['fibirllation', 'fibrilation', 'fibrilla', 'fibrillation']}, {'fibroma': ['fibroma', 'fibromialyia', 'fibromyalgia', 'fibromylagia']}]

In [59]:
simmilar_words3 = [{'fibrosing': ['fibrosing', 'fibrosis']}, {'flaygel': ['flaygel', 'flaygyll']}, {'flexaril': ['flexaril', 'flexeril']}, {'floroquinolines': ['floroquinolines', 'floroquinolins']}, {'fluorescein': ['fluorescein', 'fluorescens', 'fluorescent']}, {'fluoro': ['fluoro', 'fluoroquinolone', 'fluoroquinolones', 'fluorouracil']}, {'fluoxetine': ['fluoxetine', 'fluoxitine']}, {'fluticasone': ['fluticasone', 'fluticazolone']}, {'gabapentin': ['gabapentin', 'gabapetin']}, {'gabipentin': ['gabipentin', 'gabipenton', 'gabupentin']}, {'gadolineum': ['gadolineum', 'gadolinium']}, {'gastrocnemius': ['gastrocnemius', 'gastroeneteritis', 'gastroenteritis']}, {'gastroesophageal': ['gastroesophageal', 'gastroesophegeal', 'gastrographin', 'gastrointestil', 'gastroparesis', 'gastroparisis', 'gastrophoresis']}, {'gluten': ['gluten', 'glutete', 'gluteus']}, {'glycol': ['glycol', 'glycoprotein']}, {'haschimoto': ['haschimoto', 'hashimoto', 'hashimotos', 'hashimottos', 'hashitomotos', 'hashmotos', 'hasimotois']}, {'headache': ['headache', 'headahce', 'headedness']}, {'hemahromacytosis': ['hemahromacytosis', 'hemangioma', 'hemaplegia', 'hemarthrosis', 'hematemesis', 'hematoma', 'hematospermia', 'hematuria']}, {'hemi': ['hemi', 'hemicolectomy', 'hemicrania', 'hemiparesis', 'hemiplegia', 'hemiplegic']}, {'hemochromatosis': ['hemochromatosis', 'hemochromotosis']}, {'hemorrhage': ['hemorrhage', 'hemorrhagic', 'hemorrhoid']}, {'hemoglobin': ['hemoglobin', 'hemoglobinuria']}, {'hemodialyis': ['hemodialyis', 'hemodialysis']}, {'hepatitis': ['hepatitis', 'hepatits']}, {'hoshimoto': ['hoshimoto', 'hoshimotos', 'hosimitos']}, {'hydrochlorathiazide': ['hydrochlorathiazide', 'hydrochloride', 'hydrochlorithiazide', 'hydrochlorothiazide', 'hydrochlorthiazide', 'hydrochodone', 'hydrochorothiazide', 'hydroclorothiazide']}, {'hydrocodine': ['hydrocodine', 'hydrocodone', 'hydrocone', 'hydrocortisone']}, {'hydroxychloroquie': ['hydroxychloroquie', 'hydroxychloroquine', 'hydroxychlorquine', 'hydrozychloroquine']}, {'hyperacuasis': ['hyperacuasis', 'hyperacusis']}, {'hyperaldosternoism': ['hyperaldosternoism', 'hyperaldosteronism']}, {'hyperchloesterolemia': ['hyperchloesterolemia', 'hyperchol', 'hypercholestemia', 'hypercholesteremia', 'hypercholesterolemi', 'hypercholesterolemia', 'hypercholesterolimia', 'hypercholestrolemia']}, {'hyperhomocysteinemia': ['hyperhomocysteinemia', 'hyperhomocystinemia']}, {'hyperlidipdemia': ['hyperlidipdemia', 'hyperlipdemia', 'hyperlipemia', 'hyperlipidaemia', 'hyperlipideia', 'hyperlipidema', 'hyperlipidemis', 'hyperlipiden', 'hyperlipids']}, {'hypermetropia': ['hypermetropia', 'hypermobile', 'hypermobility']}, {'hyperosmoloty': ['hyperosmoloty', 'hyperostosis']}, {'hyperparathyroidism': ['hyperparathyroidism', 'hyperparthyrodism']}, {'hyperpotassemia': ['hyperpotassemia', 'hyperprolactemia', 'hyperprolactinemia']}, {'hypertenision': ['hypertenision', 'hypertennsion', 'hypertension', 'hypertensive', 'hypertenstion', 'hypertention', 'hypertesnion', 'hypertnesion', 'hypetension', 'hytertension']}, {'hypertriclyceridemia': ['hypertriclyceridemia', 'hypertriglycerademia', 'hypertriglyceridaemia', 'hypertriglyceridemia', 'hypogammaglobuliemia', 'hypogammaglobulinemia',
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           'inflamation', 'inflammation', 'inflammatories', 'infmamation']}, {'hypertrophic': ['hypertrophic', 'hypertrophy', 'hypoglycemia', 'hypoglycemic']}, {'hyperurecemia': ['hyperurecemia', 'hyperuricemia', 'hypokalemia', 'hypokalmia']}, {'hypomagnesemia': ['hypomagnesemia', 'hypomagnesia']}, {'hypothridism': ['hypothridism', 'hypothroidism', 'hypothryodism', 'hypothryoid', 'hypothryroid', 'hypothydriodism', 'hypothyrodism', 'hypothyroid', 'hypothyroidisn', 'hypothyroidontism']}, {'hypoxemia': ['hypoxemia', 'hypoxia', 'hypoxic', 'hypoxy', 'leukaemia', 'leukemia', 'leukoaraiosis', 'leukocyte', 'leukocytosis', 'leukopenia']}, {'lisiniopril': ['lisiniopril', 'lisinipril', 'lisinopril', 'lisinopryl', 'lisonipril']}, {'lipedema': ['lipedema', 'lipidemia']}, {'lorazepam': ['lorazepam', 'lorazepan']}, {'lymph': ['lymph', 'lymphadema', 'lymphadenopathy', 'lymphangioleiomyomatosis', 'lymphedema', 'lymphedma', 'lymphoangioleiomymatosis', 'lymphocytosis', 'lymphodema', 'lymphohistiocytosis', 'lymphoid', 'lymphoma']}, {'methylchloroisothiazinoline': ['methylchloroisothiazinoline', 'methyldibromoglutaronitrile']}, {'microademnoma': ['microademnoma', 'microadenoma']}, {'miggraine': ['miggraine', 'migraine', 'migrainosus', 'migrains', 'migrane', 'migranes', 'migrans']}, {'oropharangeal': ['oropharangeal', 'oropharyngeal']}, {'osteo': ['osteo', 'osteoarthritics', 'osteoarthritis', 'osteoarthrosis', 'osteochondritis', 'osteodystrophy', 'osteodystrpphy', 'osteomyelitis', 'osteopenia', 'osteoperosis', 'osteopetrosis', 'osteoporasis', 'osteoporisis', 'osteoporsis', 'osteosrthritis', 'osteroarthritis', 'osteroporosis']}, {'pancolitis': ['pancolitis', 'pancreas', 'pancreatic', 'pancreatitis']}, {'pencilin': ['pencilin', 'pencillin', 'pencillins', 'penecilin', 'penecillan', 'penecillin', 'penicciclin', 'peniccilin', 'peniccillin', 'penicellin', 'peniciilin', 'penicilin', 'penicillan', 'penicillans', 'penicillen', 'penicillian', 'penicilliin', 'penicillin', 'penicilline', 'penicllin', 'penincillin', 'pennecillin', 'pennicillan', 'pennicillen', 'pennicillin', 'pennicillink', 'pennicillins', 'pennicillun', 'pennicllin']}, {'pharyngoesophageal': ['pharyngoesophageal', 'pharyngoesphageal']}, {'phenegran': ['phenegran', 'phenegren', 'phenegrin', 'phenelzine', 'phenergan', 'phenergin', 'phenethyl']}, {'phenobarbital': ['phenobarbital', 'phenobarbitol']}, {'phenothiazine': ['phenothiazine', 'phenothyazines', 'phenotype']}, {'pnemonia': ['pnemonia', 'pneumomia', 'pneumonia', 'pneumonitis', 'pneumothoracic', 'pneumothorax', 'pneumovax', 'pnuemonia']}, {'prednison': ['prednison', 'prednisone', 'prednizone', 'prenisone', 'prenizone']}, {'pseudephedrine': ['pseudephedrine', 'pseudobrofimide', 'pseudobulbar']}, {'pseudoephedrine': ['pseudoephedrine', 'pseudoephidrine']}, {'pseudoseizure': ['pseudoseizure', 'pseudoseizures']}, {'thiermosol': ['thiermosol', 'thimerasol', 'thimerisol', 'thimerosal', 'thimerosol', 'thimersol']}, {'thyroidectomy': ['thyroidectomy', 'thyroidemctomy', 'thyroidism', 'thyroiditis', 'thyroidits', 'thyroidproblem', 'thyroxine']}, {'waldenstrom': ['waldenstrom', 'waldenström', 'waldonstrom']}, {'symptoms': ['symproms', 'symptom', 'symptomatic', 'symptoms', 'symtoms', 'symtpoms']}]

In [60]:
# function for replacing the words in the above lists 

def change_words(word):
    for dic in simmilar_words:
        for key, values in dic.items():
            if word in values:
                return key
    return word


def change_words2(word):
    for dic in simmilar_words2:
        for key, values in dic.items():
            if word in values:
                return key
    return word


def change_words3(word):
    for dic in simmilar_words3:
        for key, values in dic.items():
            if word in values:
                return key
    return word

In [61]:
# apply the change_word functions 
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [change_words(word) for word in x])
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [change_words2(word) for word in x])
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [change_words3(word) for word in x])

In [62]:
meaningless = ['comment', 'history', 'patient', 'history', 'aaa', 'ab', 'abd', 'abt', 'ac', 'ace', 'acf', 'acid', 'acne', 'acre', 'add', 'adh', 'adm', 'adn', 'ae', 'af', 'afi', 'agar', 'age', 'ahd', 'air', 'aka', 'aki', 'al', 'alan', 'alex', 'alf', 'ali', 'alk', 'all', 'alla', 'alma', 'alot', 'als', 'alt', 'am', 'amd', 'amor', 'amp', 'ams', 'ance', 'ang', 'anl', 'ant', 'any', 'ap', 'apc', 'ape', 'apex', 'app', 'apps', 'apr', 'aps', 'arb', 'arch', 'arcs', 'are', 'area', 'arf', 'aria', 'arid', 'arm', 'arms', 'army', 'arts', 'asa', 'asap', 'asd', 'ash', 'asia', 'ast', 'atb', 'ate', 'atom', 'au', 'aug', 'auto', 'av', 'avg', 'awa', 'az', 'b', 'ba', 'back', 'bag', 'ban', 'band', 'bar', 'bare', 'bark', 'barr', 'bars', 'bas', 'base', 'bav', 'bay', 'bbb', 'bc', 'bcc', 'bed', 'been', 'bg', 'bh', 'bha', 'bhp', 'bi', 'bid', 'bil', 'bio', 'bit', 'bk', 'ble', 'bmi', 'bms', 'bnp', 'bot', 'bp', 'br', 'bs', 'bss', 'bug', 'bun', 'bus', 'but', 'c', 'ca', 'cab', 'cad', 'can', 'cap', 'car', 'cas', 'cat', 'cbc', 'cbs', 'cc', 'cce', 'ccm', 'cd', 'cf', 'ch', 'chg', 'chi', 'chr', 'ci', 'cic', 'cid', 'cin', 'cir', 'cis', 'ciu', 'ck', 'cko', 'cl', 'cm', 'cmc', 'cmo', 'cmp', 'cms', 'cmt', 'col', 'con', 'cor', 'cow', 'cp', 'cr', 'crc', 'cri', 'cry', 'cs', 'ct', 'cta', 'cts', 'cum', 'cut', 'cv', 'cvs', 'da', 'dad', 'day', 'dbs', 'dcv', 'ddd', 'de', 'dec', 'del', 'des', 'dhe', 'di', 'dia', 'did', 'die', 'dif', 'dip', 'dir', 'dis', 'div', 'dl', 'dm', 'dn', 'dnr', 'doe', 'dog', 'dot', 'dp', 'dr', 'dry', 'ds', 'du', 'duo', 'dur', 'dx', 'dye', 'dyn', 'dz', 'e', 'ear', 'eat', 'eb', 'ed', 'eds', 'ees', 'ef', 'ehr', 'em', 'ems', 'end', 'eng', 'ent', 'epi', 'er', 'erm', 'esp', 'esr', 'et', 'eta', 'etc', 'eth', 'exp', 'f', 'fa', 'fat', 'fe', 'feb', 'fil', 'fir', 'fix', 'fld', 'flu', 'fmf', 'fms', 'fr', 'fur', 'fx', 'g', 'ga', 'gad', 'gag', 'gal', 'gb', 'gbs', 'ge', 'gel', 'gen', 'ger', 'get', 'gi', 'gm', 'gol', 'got', 'gp', 'gt', 'gum', 'gun', 'gut', 'h', 'ha', 'had', 'hay', 'hb', 'hbo', 'hc', 'hd', 'hdl', 'heb', 'hep', 'hf', 'hh', 'hi', 'hip', 'hit', 'hiv', 'hl', 'ho', 'hop', 'how', 'hp', 'hpt', 'hr', 'hrs', 'hrt', 'hsi', 'hsv', 'ht', 'htm', 'hx', 'ia', 'ic', 'ica', 'icd', 'ice', 'icu', 'id', 'ida', 'idd', 'idm', 'idp', 'if', 'ig', 'iga', 'ige', 'igm',
               'ii', 'iii', 'il', 'ill', 'ils', 'im', 'img', 'in', 'inh', 'inj', 'ink', 'int', 'ip', 'iq', 'is', 'it', 'iu', 'iv', 'ivi', 'ivy', 'j', 'jan', 'jaw', 'job', 'jul', 'k', 'ka', 'kd', 'kg', 'kid', 'kts', 'ku', 'l', 'la', 'lab', 'lad', 'lap', 'lb', 'lbs', 'le', 'leg', 'let', 'lip', 'll', 'llt', 'lon', 'lot', 'lp', 'lpg', 'lt', 'ltf', 'lv', 'ma', 'mac', 'mag', 'mai', 'mal', 'man', 'map', 'mar', 'max', 'mc', 'mca', 'mcd', 'mcg', 'mcs', 'md', 'mdd', 'mdr', 'mds', 'mes', 'mg', 'mgm', 'mi', 'mid', 'mil', 'mix', 'ml', 'mla', 'mm', 'mo', 'mod', 'mom', 'mos', 'mpa', 'mph', 'mr', 'ms', 'msa', 'msg', 'mst', 'mus', 'mvc', 'mvp', 'mw', 'n', 'nc', 'nd', 'nda', 'neg', 'nfa', 'nh', 'nhl', 'nid', 'nil', 'nk', 'nka', 'nma', 'nn', 'non', 'nor', 'nos', 'nov', 'npo', 'ns', 'nt', 'nta', 'num', 'nut', 'nya', 'oa', 'oak', 'oat', 'occ', 'oct', 'od', 'oil', 'ol', 'old', 'om', 'one', 'op', 'opa', 'os', 'osa', 'oth', 'ou', 'out', 'ox', 'oxy', 'p', 'pa', 'pab', 'pad', 'pai', 'pak', 'pap', 'pat', 'pay', 'pb', 'pd', 'pda', 'pdf', 'pe', 'pea', 'pec', 'peg', 'pen', 'per', 'pes', 'pet', 'pf', 'pfo', 'pfr', 'ph', 'phx', 'pid', 'pig', 'pit', 'pm', 'pmc', 'pmn', 'pmr', 'pms', 'po', 'poc', 'pog', 'pos', 'pox', 'pp', 'ppi', 'ppm', 'ppv', 'pr', 'prd', 'pre', 'prn', 'pro', 'psa', 'psp', 'pt', 'pts', 'put', 'pvt', 'q', 'qd', 'qt', 'r', 'ra', 'rad', 'rag', 'rah', 'rat', 'ray', 'rca', 'rec', 'red', 'rel', 'ret', 'rev', 'rib', 'rka', 'rle', 'rls', 'rn', 'rod', 'rom', 'row', 'rr', 'rt', 'rue', 'run', 'rv', 'rx', 'ry', 'rye', 'sal', 'sap', 'sar', 'saw', 'sbe', 'sbl', 'scc', 'scp', 'scr', 'sct', 'sea', 'see', 'sfn', 'sh', 'sha', 'she', 'sio', 'ski', 'so', 'sob', 'sol', 'son', 'soy', 'sp', 'spi', 'sr', 'sri', 'srs', 'ss', 'sss', 'st', 'stg', 'sua', 'sud', 'sun', 'sup', 'sur', 'svt', 'sw', 'sx', 'syn', 'syr', 'tab', 'tap', 'tar', 'tb', 'tbd', 'td', 'te', 'tea', 'tee', 'ten', 'th', 'tho', 'thr', 'ti', 'tia', 'tid', 'til', 'tka', 'tm', 'tn', 'toe', 'tos', 'tr', 'try', 'tss', 'tst', 'tts', 'tu', 'tum', 'tx', 'ua', 'uk', 'ul', 'unk', 'ups', 'uri', 'url', 'use', 'utc', 'uti', 'v', 'va', 'vac', 'vet', 'via', 'vid', 'vio', 'vit', 'vk', 'von', 'vp', 'vs', 'vt', 'w', 'war', 'was', 'way', 'wbc', 'well', 'will', 'wit', 'with', 'wu', 'www', 'x', 'xl', 'yes', 'yo', 'you', 'yrs', 'z']
# remove meaning less words 
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [word for word in x if word not in meaningless])

In [67]:
allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
allwords_list = list(allwords)
allwords = sorted(allwords_list)
len(allwords)

4938

In [69]:
def get_wordnet_pos(tag):
    x, y = tag[0]
    if y.startswith('J'):
        return wordnet.ADJ
    elif y.startswith('V'):
        return wordnet.VERB
    elif y.startswith('N'):
        return wordnet.NOUN
    elif y.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [70]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from nltk import pos_tag
st = PorterStemmer()
wnl = WordNetLemmatizer()

### After all the data cleanings, the number of unique keywords related to the patients' health is reduced from 16361 to 4517

In [71]:
df['HEALTH_CONDITION'] = df['HEALTH_CONDITION'].apply(
    lambda x: [wnl.lemmatize(w, pos=get_wordnet_pos(pos_tag([w]))) for w in x])


allwords = set(itertools.chain.from_iterable(df.HEALTH_CONDITION))
len(allwords)

4517

In [73]:

df_training = df

# convert the lists in health condition to string
df_training['HEALTH_CONDITION'] = df_training['HEALTH_CONDITION'].apply(
    lambda x: " ".join(x))
df_training.head()

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS,AGE_CAT,HEALTH_CONDITION
0,916600,33.0,F,,,pcn and bee venom,MODERNA,"[Dysphagia, Epiglottitis]",20-39 yr,bee venom
1,916601,73.0,F,patient residing at nursing facility. see pati...,patient residing at nursing facility. see pati...,"""dairy""",MODERNA,"[Anxiety, Dyspnoea]",over_60 yr,facility patient nursing facility patient
2,916602,23.0,F,,,shellfish,PFIZER,"[Chest discomfort, Dysphagia, Pain in extremit...",20-39 yr,
3,916603,58.0,F,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","diclofec, novacaine, lidocaine, pickles, tomat...",MODERNA,"[Dizziness, Fatigue, Mobility decreased]",40-59 yr,kidney infection diverculitis valve relapse os...
4,916604,47.0,F,,,,MODERNA,"[Injection site erythema, Injection site pruri...",40-59 yr,


# Model 

In [74]:
MODERNA = df_training[df_training['VAX_NAME'] == 'MODERNA']
PFIZER = df_training[df_training['VAX_NAME'] == 'PFIZER']

### Split datasets for training and testing 

In [75]:
from sklearn.model_selection import train_test_split
MODERNA_train, MODERNA_test = train_test_split(
    MODERNA, stratify=MODERNA['AGE_CAT'], test_size=0.05)
PFIZER_train, PFIZER_test = train_test_split(
    PFIZER, stratify=PFIZER['AGE_CAT'], test_size=0.05)

### Vectorize the training set 

In [76]:
# make df_training data to sparse matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
# define stop words
stop_w = text.ENGLISH_STOP_WORDS


def vectorizing(data_train):
    tfidf_vectorizer = TfidfVectorizer(stop_words=stop_w)
    ohe = OneHotEncoder(handle_unknown='ignore')

    tfidfs = tfidf_vectorizer.fit_transform(data_train['HEALTH_CONDITION'])
    tfidfs = pd.DataFrame(
        tfidfs.toarray(), columns=tfidf_vectorizer.get_feature_names())

    dummie_age_sex = ohe.fit_transform(data_train[['SEX', 'AGE_CAT']])
    dummie_age_sex = pd.DataFrame(
        dummie_age_sex.toarray(), columns=ohe.get_feature_names())
    vector = pd.concat([dummie_age_sex, tfidfs], axis=1)
    return {'vector': vector, 'tfidf': tfidf_vectorizer, "ohe": ohe}

In [77]:
moderna_vectorizer = vectorizing(MODERNA_train)
pfizer_vectorizer = vectorizing(PFIZER_train)

## Moderna K-means model training

In [79]:
from sklearn.cluster import KMeans
N_CLUSTER = 100
kmeans_MODERNA = KMeans(n_clusters=N_CLUSTER)
kmeans_MODERNA.fit(moderna_vectorizer.get('vector'))
MODERNA_train = MODERNA_train.copy()
MODERNA_train['CLUSTER'] = kmeans_MODERNA.labels_

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=40)
kmeans.fit(moderna_vector)

## Pfizer K-means model training¶

In [80]:
kmeans_PFIZER = KMeans(n_clusters=N_CLUSTER)
kmeans_PFIZER.fit(pfizer_vectorizer.get('vector'))
PFIZER_train = PFIZER_train.copy()
PFIZER_train['CLUSTER'] = kmeans_PFIZER.labels_

### Number of instance in each cluster - Moderna

In [81]:
moderna_group = MODERNA_train.groupby(
    by='CLUSTER')['VAERS_ID'].agg('count').sort_values().to_frame()
moderna_group.columns = ['num of sample']
moderna_group

,num of sample
CLUSTER,
4,1
5,1
15,1
81,2
72,2
94,2
93,4
91,4
78,4


### Cluster detail

In [83]:
sym_freq = MODERNA_train[MODERNA_train['CLUSTER'] == 95]
sym_freq=sym_freq.drop(['CUR_ILL','HISTORY','ALLERGIES'], axis=1)
sym_freq.head()

,VAERS_ID,AGE_YRS,SEX,VAX_NAME,SYMPTOMS,AGE_CAT,HEALTH_CONDITION,CLUSTER
2168,919106,59.0,F,MODERNA,"[Chills, Drug eruption, Pain, Pain in extremit...",40-59 yr,cancer chem,95
14132,931150,53.0,F,MODERNA,"[Decreased appetite, Fatigue, Feeling hot, Hyp...",40-59 yr,breast cancer,95
33312,1087235,44.0,F,MODERNA,[Lymphoedema],40-59 yr,breast cancer remission coconut antibiotic sul...,95
16275,932826,51.0,F,MODERNA,"[Arthralgia, Asthenia, Chills, Fatigue, Headac...",40-59 yr,stage colon cancer,95
5066,922450,44.0,F,MODERNA,"[Injection site erythema, Injection site pruri...",40-59 yr,chemo cancer remission,95


###  Symptoms frequency 

In [84]:
from itertools import chain
from collections import Counter
modern_group_sym_fre = pd.Series(Counter(chain.from_iterable(
    sym_freq.SYMPTOMS.dropna()))).sort_values(ascending=False).head(15).to_frame()
modern_group_sym_fre.columns = ['Frequency']
modern_group_sym_fre

,Frequency
Chills,11
Headache,9
Pain,8
Fatigue,8
Pyrexia,8
Pain in extremity,6
Pruritus,6
Injection site erythema,6
Erythema,5
Dizziness,4


#### ELBOW METHOD

k = [x for x in range(5,50) if x%5==0]


wss=[]
for i in k:
    print(i)
    kmeans=KMeans(n_clusters=i, init='k-means++')
    kmeans=kmeans.fit(moderna_vectorizer.get('vector'))
    wss_iter=kmeans.inertia_
    wss.append(wss_iter)

elbow=pd.DataFrame({'#Cluster':k , 'WSS': wss})

elbow

import seaborn as sns 
from matplotlib import pyplot as plt
sns.scatterplot(x=elbow['#Cluster'], y=elbow['WSS'], marker="+").set_title('MODERNA')

wss_pfizer=[]

for i in k:
    print(i)
    kmeans=KMeans(n_clusters=i, init='k-means++')
    kmeans=kmeans.fit(moderna_vectorizer.get('vector'))
    wss_iter=kmeans.inertia_
    wss_pfizer.append(wss_iter)

elbow_pfizer=pd.DataFrame({'#Cluster':k , 'WSS': wss_pfizer})

sns.scatterplot(x=elbow_pfizer['#Cluster'], y=elbow_pfizer['WSS'], marker="+").set_title('PFIZER')

### Number of instance in each cluster - Pfizer

In [88]:
pfizer_group=PFIZER_train.groupby(by='CLUSTER')['VAERS_ID'].agg('count').sort_values().to_frame()
pfizer_group.columns = ['Frequency']
pfizer_group

,Frequency
CLUSTER,
91,2
66,2
73,2
36,3
49,4
27,5
94,5
21,5
55,5


###  Cluster detail

In [95]:
sym_freq = PFIZER_train[PFIZER_train['CLUSTER'] == 89]
sym_freq.head()

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS,AGE_CAT,HEALTH_CONDITION,CLUSTER
19870,975184,89.0,F,,heart disease mild stroke about a year ago,,PFIZER,"[Death, Myocardial infarction]",over_60 yr,heart disease mild year,89
28847,1059621,71.0,F,"obesity, diabetes type 2, heart failure, bipol...",see above,"daptomycin, penicillin, vancomycin, cipro, cub...",PFIZER,"[COVID-19, Chest X-ray normal, Death, Dyspnoea...",over_60 yr,obesity heart failure edema cardia penicillin ...,89
12645,930037,64.0,F,no,heart disease heart stint,no,PFIZER,"[Dyspepsia, Injection site erythema, Injection...",over_60 yr,heart heart stint,89
33855,1093698,64.0,F,,"cad, dm2, cva, seizure disorder, pseudoseizure...",codeine,PFIZER,"[Electroencephalogram, Seizure, Seizure like p...",over_60 yr,seizure disorder anxiety polio combine,89
20860,993112,62.0,F,broken ankle,"mid intellectual disabilities, schizoaffective...","pcn,lactose intol",PFIZER,"[Aphasia, Cardiac arrest, Death, Decreased app...",over_60 yr,ankle disability disorder heart disease hypert...,89


### Symptoms frequency 

In [98]:
from itertools import chain
from collections import Counter
pfizer_group_sym_fre=pd.Series(Counter(chain.from_iterable(sym_freq.SYMPTOMS.dropna()))
          ).sort_values(ascending=False).head(10).to_frame()

pfizer_group_sym_fre.columns = ['Frequency']
pfizer_group_sym_fre

,Frequency
Death,32
Headache,25
Nausea,24
Pyrexia,20
Fatigue,20
Pain,17
Dizziness,15
Dyspnoea,14
Chills,13
Asthenia,13


# Model Evaluation

In [100]:
# function for vectorize and predict cluster labels for the test set 
def get_prediction(new_sample, vectorizer, model):
    # one hot encode [sex, age_cat]
    sample_dummi = vectorizer.get("ohe").transform(
        new_sample[['SEX', 'AGE_CAT']])
    sample_dummi = pd.DataFrame(sample_dummi.toarray(
    ), columns=vectorizer.get("ohe").get_feature_names())

    # vectorize the new sample

    sample_tfidf = vectorizer.get("tfidf").transform(
        new_sample['HEALTH_CONDITION'])
    sample_tfidf = pd.DataFrame(sample_tfidf.toarray(
    ), columns=vectorizer.get("tfidf").get_feature_names())

    # combine tfidf and dummie to one df and use it as a parameter for the kmeans prediction
    sampleData = pd.concat([sample_dummi, sample_tfidf], axis=1)

    prediction = model.predict(sampleData)
    return prediction

In [101]:
# function for getting the top 10 symptoms in each cluster of the trainning set
def get_top10_sym_per_cluster(data_train, number_cluster):
    data_by_cluster = {}
    for i in range(N_CLUSTER):
        cluster = data_train[data_train['CLUSTER'] == i]
        cluster = pd.Series(Counter(chain.from_iterable(
            cluster.SYMPTOMS.dropna()))).sort_values(ascending=False).head(10).to_frame()
        cluster.reset_index()
        cluster.columns = ['qty']
        data_by_cluster[i] = cluster
    return data_by_cluster

# function to compute what % of the test sample's actual symptoms are matching with the top 10 symptoms,  
def get_symptom_accuracy_count(data_test, top_10symp_by_cluster):
    MATCHING = []
    TOP10=[]
    for index, row in data_test.iterrows():
        count = 0
        for word in row['SYMPTOMS']:
            if word in top_10symp_by_cluster.get(row.PREDICTION).index.tolist():
                count = count+1
        MATCHING.append(count/len(row['SYMPTOMS']))
        TOP10.append(top_10symp_by_cluster.get(row.PREDICTION).index.tolist())
    data_test['MATCHING'] = MATCHING
    data_test['TOP10_SYMP']=TOP10
    return data_test

In [105]:
# make a copy of the test sets
MODERNA_test = MODERNA_test.copy()
PFIZER_test = PFIZER_test.copy()

# get cluster labels for the test sets
MODERNA_test['PREDICTION'] = get_prediction(
    MODERNA_test, moderna_vectorizer, kmeans_MODERNA)
PFIZER_test['PREDICTION'] = get_prediction(
    PFIZER_test, pfizer_vectorizer, kmeans_PFIZER)

# get the 10 symptoms of each cluster in the training sets and store them in dictionaries
top10_symp_moderna = get_top10_sym_per_cluster(MODERNA_train, N_CLUSTER)
top10_symp_pfizer = get_top10_sym_per_cluster(PFIZER_train, N_CLUSTER)

# for each instance in the test set, find out how many of its symptoms are matching with the top 10 symptoms of the same cluster in the training set
MODERNA_test = get_symptom_accuracy_count(MODERNA_test, top10_symp_moderna)
PFIZER_test = get_symptom_accuracy_count(PFIZER_test, top10_symp_pfizer)

In [107]:
MODERNA_test[['AGE_CAT','SEX','VAX_NAME','AGE_CAT','HEALTH_CONDITION','SYMPTOMS','PREDICTION','MATCHING','TOP10_SYMP']].head(5)

,AGE_CAT,SEX,VAX_NAME,AGE_CAT,HEALTH_CONDITION,SYMPTOMS,PREDICTION,MATCHING,TOP10_SYMP
26393,40-59 yr,F,MODERNA,40-59 yr,,"[Chills, Ear discomfort, Headache, Hyperhidros...",1,0.500000,"[Headache, Pyrexia, Chills, Injection site ery..."
28241,over_60 yr,M,MODERNA,over_60 yr,disease depression hyperchloesterolemia hypert...,"[Death, Injection site pain]",50,0.500000,"[Death, Asthenia, Pyrexia, Dyspnoea, Chills, A..."
6897,40-59 yr,F,MODERNA,40-59 yr,arteriole hyper mobility migrant mushroom,"[Blood test, Dizziness, Flushing, Nausea, Orop...",1,0.285714,"[Headache, Pyrexia, Chills, Injection site ery..."
25140,over_60 yr,M,MODERNA,over_60 yr,,"[Asthenia, Fatigue, Nausea, Pain in extremity,...",2,0.600000,"[Death, Pyrexia, Chills, Fatigue, Headache, My..."
12886,over_60 yr,F,MODERNA,over_60 yr,hypertenision,"[Arthralgia, Erythema, Fatigue, Induration, Lo...",29,0.000000,"[Injection site erythema, Pyrexia, Chills, Hea..."


In [108]:
PFIZER_test[['AGE_CAT','SEX','VAX_NAME','AGE_CAT','HEALTH_CONDITION','SYMPTOMS','PREDICTION','MATCHING','TOP10_SYMP']].head(5)

,AGE_CAT,SEX,VAX_NAME,AGE_CAT,HEALTH_CONDITION,SYMPTOMS,PREDICTION,MATCHING,TOP10_SYMP
16351,20-39 yr,F,PFIZER,20-39 yr,aasthma avalox bee,"[Arthralgia, Chills, Fatigue, Injection site r...",22,0.500000,"[Headache, Pyrexia, Fatigue, Pain, Chills, Nau..."
18677,20-39 yr,F,PFIZER,20-39 yr,,[Unevaluable event],6,0.000000,"[Headache, Chills, Pyrexia, Fatigue, Pain, Nau..."
2549,20-39 yr,M,PFIZER,20-39 yr,,"[Anxiety, Tachycardia, Tremor]",1,0.000000,"[Headache, Pyrexia, Chills, Fatigue, Pain, Mya..."
4783,40-59 yr,F,PFIZER,40-59 yr,,"[Blood pressure increased, Flushing, Throat ir...",4,0.000000,"[Headache, Pyrexia, Fatigue, Chills, Pain, Nau..."
28634,over_60 yr,F,PFIZER,over_60 yr,alergies aluminium bee hypersensitivity hypert...,"[Chills, Erythema, Eye swelling, Feeling hot, ...",53,0.333333,"[Headache, Chills, Fatigue, Nausea, Dizziness,..."


### Model evaluation result 

In [110]:
print("Modern's model accuracy is " + str(MODERNA_test['MATCHING'].mean()))
print("Pfizer's model accuracy is " + str(PFIZER_test['MATCHING'].mean()))

Modern's model accuracy is 0.34100033274940017
Pfizer's model accuracy is 0.3247223530862766


### Export the models, onehotencoders, and Tifidf vectorizers 

In [125]:
from joblib import dump
dump(kmeans_MODERNA, './exported models/kmeans_MODERNA_model.joblib')
dump(kmeans_PFIZER, './exported models/kmeans_PFIZER_model.joblib')
dump(moderna_vectorizer.get('ohe'), './exported models/onehotendcoder_moderna.joblib')
dump(pfizer_vectorizer.get("ohe"), './exported models/onehotendcoder_pfizer.joblib')
dump(moderna_vectorizer.get('tfidf'), './exported models/tfidf_vectorizer_moderna.joblib')
dump(pfizer_vectorizer.get("tfidf"), './exported models/tfidf_vectorizer_pfizer.joblib')

['./exported models/tfidf_vectorizer_pfizer.joblib']

### Export the training dataframe with cluster labels

In [126]:
MODERNA_train.to_pickle("./exported models/MODERNA_DF.pkl")
PFIZER_train.to_pickle("./exported models/PFIZER_DF.pkl")